In [1]:
import docker
print("imported docker successfully.")

imported docker successfully.


In [2]:
cont=['hive-server','namenode','datanode','historyserver','hive-metastore-postgresql'
,'presto-coordinator','hive-metastore','nodemanager','resourcemanager']

Stopping all the running containers

In [3]:
client=docker.from_env()
for container in client.containers.list():
    print("container:{} stopped.".format(container.name))
    container.stop()


container:datanode stopped.
container:historyserver stopped.
container:resourcemanager stopped.
container:namenode stopped.
container:nodemanager stopped.


Starting required containers


In [4]:
for container in client.containers.list(True):
    if container.name in cont:
        print("container:{} started.".format(container.name))
        container.start()
print("all live containers:")
for container in client.containers.list():
    print(container.name,container.id)

container:hive-server started.
container:datanode started.
container:historyserver started.
container:hive-metastore-postgresql started.
container:resourcemanager started.
container:namenode started.
container:nodemanager started.
container:presto-coordinator started.
container:hive-metastore started.
all live containers:
hive-server 0a705452ffd0fb47a60689d581e2d6e38c4fb82a6d260d379f9aee646f88e4b4
datanode 090a9d330f84cc4a6968e4f9a5e2e65dbff558566749fb63b35710a4f193d0fa
historyserver 695c419b5345a90269822a6d327a14234c6ffbc1934afd68593b18d244556868
hive-metastore-postgresql b53bcda67f391c98538477775d3ecd9a3b08f614d80498befd1046e6d3bf453f
resourcemanager a3ab9e4776cc91f45ddc1af40618ad972b670a81f3ba07fc0c5845e1877b00c7
namenode 3014cb7bf7745fc2b35ab7286eafed5b4246897ca32dd3ee55783631116714e3
nodemanager 86fe60ec723efbdfbf5e32f3dd3504a7487e48a370637fe5ec06e27fc95afecd
presto-coordinator 82580ab4c7e8faec3e4c3006f43005a915ab52aaa628f345fdd7e3310eba9e82
hive-metastore 8e7b59c62861435d21782961

In [5]:
cobj=[]
for container in client.containers.list():
    if container.name == 'hive-server':
        cobj.append(container)
i=str(cobj[0].exec_run('hdfs dfs -ls /')).split('\\n')
for p in i:
    print(p)


ExecResult(exit_code=0, output=b'Found 4 items
drwxr-xr-x   - root supergroup          0 2023-05-11 19:27 /data
drwxr-xr-x   - root supergroup          0 2023-04-25 08:07 /rmstate
drwxrwxr-x   - root supergroup          0 2023-04-25 20:35 /tmp
drwxr-xr-x   - root supergroup          0 2023-04-25 20:35 /user
')


In [7]:
type(cobj[0])

docker.models.containers.Container

In [8]:
for p in i[0].split('\\n'):
    print(p)
    #print("hello")

ExecResult(exit_code=0, output=b'Found 4 items


In [9]:
cobj[0].exec_run("hive -e 'set hive.cli.print.header=true;'")

ExecResult(exit_code=0, output=b'SLF4J: Class path contains multiple SLF4J bindings.\nSLF4J: Found binding in [jar:file:/opt/hive/lib/log4j-slf4j-impl-2.6.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]\nSLF4J: Found binding in [jar:file:/opt/hadoop-2.7.4/share/hadoop/common/lib/slf4j-log4j12-1.7.10.jar!/org/slf4j/impl/StaticLoggerBinder.class]\nSLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.\nSLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]\n\nLogging initialized using configuration in file:/opt/hive/conf/hive-log4j2.properties Async: true\n')

In [15]:
#DROP TABLES
i=str(cobj[0].exec_run\
("""hive -e 'use hive_db; drop table sales_order_csv_orc_v1_backup'
""")).split('true')
for line in i[1].split('\\n'):
    print(line.split('\\t'))

['']
['OK']
['Time taken: 1.989 seconds']
['OK']
['Time taken: 0.166 seconds']
["')"]


In [10]:
#CREATE SUB-TABLES
cobj[0].exec_run\
("""hive -e "use hive_db; create table sales_order_csv_orc_v1_backup as 
select * from sales_order_data_csv_v1_orc where status='Shipped' limit 20;" """)

ExecResult(exit_code=0, output=b'SLF4J: Class path contains multiple SLF4J bindings.\nSLF4J: Found binding in [jar:file:/opt/hive/lib/log4j-slf4j-impl-2.6.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]\nSLF4J: Found binding in [jar:file:/opt/hadoop-2.7.4/share/hadoop/common/lib/slf4j-log4j12-1.7.10.jar!/org/slf4j/impl/StaticLoggerBinder.class]\nSLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.\nSLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]\n\nLogging initialized using configuration in file:/opt/hive/conf/hive-log4j2.properties Async: true\nOK\nTime taken: 6.843 seconds\nWARNING: Hive-on-MR is deprecated in Hive 2 and may not be available in the future versions. Consider using a different execution engine (i.e. spark, tez) or using Hive 1.X releases.\nQuery ID = root_20230515114822_c977d955-ec76-4346-a7c9-cea5fa6488db\nTotal jobs = 1\nLaunching Job 1 out of 1\nNumber of reduce tasks determined at compile time: 1\nIn order 

In [11]:
#FETCHING ROWS FROM A TABLE
i=str(cobj[0].exec_run\
("""hive -e "set hive.cli.print.header=true;
select * from sales_order_csv_orc where city ='San Francisco' limit 10;" """))\
.split('true')

for line in i[1].split('\\n'):
    print(line.split('\\t'))

['']
['OK']
['sales_order_csv_orc.ordernumber', 'sales_order_csv_orc.quantityordered', 'sales_order_csv_orc.priceeach', 'sales_order_csv_orc.orderlinenumber', 'sales_order_csv_orc.sales', 'sales_order_csv_orc.status', 'sales_order_csv_orc.qtr_id', 'sales_order_csv_orc.month_id', 'sales_order_csv_orc.year_id', 'sales_order_csv_orc.productline', 'sales_order_csv_orc.msrp', 'sales_order_csv_orc.productcode', 'sales_order_csv_orc.phone', 'sales_order_csv_orc.city', 'sales_order_csv_orc.state', 'sales_order_csv_orc.postalcode', 'sales_order_csv_orc.country', 'sales_order_csv_orc.territory', 'sales_order_csv_orc.contactlastname', 'sales_order_csv_orc.contactfirstname', 'sales_order_csv_orc.dealsize']
['10159', '49', '100.0', '14', '5205.27', 'Shipped', '4', '10', '2003', 'Motorcycles', '95', 'S10_1678', '6505551386', 'San Francisco', 'CA', '', 'USA', 'NA', 'Brown', 'Julie', 'Medium']
['10201', '22', '98.57', '2', '2168.54', 'Shipped', '4', '12', '2003', 'Motorcycles', '95', 'S10_1678', '6505

In [3]:
from pyhive import hive

host_name = "172.19.0.7"
port = 10000
user = "root"
password = "password"
database="hive_db"

def hiveconnection(host_name, port, user, database):
    conn = hive.Connection(host=host_name, port=port, username=user,
                           database=database)
    cur = conn.cursor()
    cur.execute("show databases;")
    result = cur.fetchall()

    return result

# Call above function
output = hiveconnection(host_name, port, user, database)
print(output)

ModuleNotFoundError: No module named 'sasl'